In [6]:
## scratch pad for citibike data set! exciting! get to break out my pandas knowledge
import pandas as pd
print(pd.__version__) #fortunately, latest version of pandas allows direct import from zip
import zipfile
import requests
import io

0.19.0


In [8]:
citidir = "https://s3.amazonaws.com/tripdata/"
citi_extension = "-citibike-tripdata.zip"
year = 2015
months = ['01','02','03','04','05','06','07','08','09','10','11','12']

In [12]:
for mth in months:
    filename = str(year) + mth + citi_extension
    citi_file = citidir + filename
    print(citi_file)
    requested_file = requests.get(citi_file)
    z = zipfile.ZipFile(io.BytesIO(requested_file.content))
    df = pd.read_csv(z.open(filename), header=1, sep=',', quotechar='"')

https://s3.amazonaws.com/tripdata/201501-citibike-tripdata.zip


KeyError: "There is no item named '201501-citibike-tripdata.zip' in the archive"

In [ ]:
for mth in months:
    citi_file = citidir + str(year) + mth + citi_extension
    print(citi_file)
    df = pd.read_csv(citi_file, compression='zip', header=1, sep=',', quotechar='"')